### The flow and description

The flow
1) Getting the data from Job Board - HH
2) Cleaning the data
3) Storing them in a single dataframe
4) Uploading the dataframe into my Database
5) Running descriptive analysis
6) Skill extraction using OpenAi ChatGPT

In [ ]:
# checkpoint
df = pd.read_csv('/Users/igorrudakov/Desktop/last_download_hh.csv')
df.head().T

In [2]:
import requests      # Для запросов по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import pandas as pd  # Для формирования датафрейма с результатами


# Getting the data from Job Boards

## Head Hunter

### Getting HeadHunter professional roles

In [5]:
# request to hh api
def getProfRoles():
    req = requests.get('https://api.hh.ru/professional_roles')
    data = req.content.decode()
    req.close()
    return data

prof_roles =  pd.read_json( getProfRoles() )
prof_roles.head()

,categories
0,"{'id': '19', 'name': 'Автомобильный бизнес', '..."
1,"{'id': '5', 'name': 'Административный персонал..."
2,"{'id': '15', 'name': 'Безопасность', 'roles': ..."
3,"{'id': '26', 'name': 'Высший и средний менеджм..."
4,"{'id': '8', 'name': 'Добыча сырья', 'roles': [..."


In [6]:
# parsing the json and creating a dataframe

df_list = []

for category in prof_roles["categories"]:
    for role in category["roles"]:
        df_list.append({
            "category_id": category["id"],
            "category_name": category["name"],
            "role_id": role["id"],
            "role_name": role["name"],
            "accept_incomplete_resumes": role["accept_incomplete_resumes"],
            "is_default": role["is_default"]
        })


prof_roles_df = pd.DataFrame(df_list)


In [7]:
# renaming the columns
prof_roles_df = prof_roles_df.rename(columns={'category_id':'parent_profession_id',
                              'category_name':'parent_profession_name'
                              })
prof_roles_df.head()

,parent_profession_id,parent_profession_name,role_id,role_name,accept_incomplete_resumes,is_default
0,19,Автомобильный бизнес,4,Автомойщик,True,False
1,19,Автомобильный бизнес,5,"Автослесарь, автомеханик",True,False
2,19,Автомобильный бизнес,62,Мастер-приемщик,True,False
3,19,Автомобильный бизнес,70,"Менеджер по продажам, менеджер по работе с кли...",False,False
4,5,Административный персонал,8,Администратор,True,False


In [8]:
# leaving only clear roles (droping the parent ids)

prof_roles_df_short = prof_roles_df[['role_id','role_name']].drop_duplicates()
prof_roles_df_short.head()

,role_id,role_name
0,4,Автомойщик
1,5,"Автослесарь, автомеханик"
2,62,Мастер-приемщик
3,70,"Менеджер по продажам, менеджер по работе с кли..."
4,8,Администратор


In [40]:
# saving to csv if needed
# prof_roles_df_short.to_csv('/Users/igorrudakov/Desktop/roles_ids.csv')

### Testing the API connection

In [405]:
# you can find any area by using filter below
for i in area_data[area_data.name == 'Россия'].areas[0]:
    if i['name'] == 'Москва':
        print(i)


{'id': '1', 'parent_id': '113', 'name': 'Москва', 'areas': []}


In [6]:
# testing
base_url = "https://api.hh.ru/vacancies"
params = {
        "text": '',         # Ключевые слова в вакансии (OR между ними)
        'per_page': 10,      # Кол-во вакансий на 1 странице
        'area': 1,
        'professional_role': '10',
        'page':1
    }
daga = requests.get(base_url, params=params)



In [7]:
#  how the data looks like
daga.json()['items'][1]

{'id': '90595126',
 'premium': False,
 'name': 'Аналитик (удаленная работа)',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': {'from': 92000, 'to': 115000, 'currency': 'RUR', 'gross': True},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Москва',
  'street': 'Калужско-Рижская линия',
  'building': 'метро Профсоюзная',
  'lat': 55.67793,
  'lng': 37.562874,
  'description': None,
  'raw': 'Москва, Калужско-Рижская линия, метро Профсоюзная',
  'metro': {'station_name': 'Профсоюзная',
   'line_name': 'Калужско-Рижская',
   'station_id': '6.121',
   'line_id': '6',
   'lat': 55.677671,
   'lng': 37.562595},
  'metro_stations': [{'station_name': 'Профсоюзная',
    'line_name': 'Калужско-Рижская',
    'station_id': '6.121',
    'line_id': '6',
    'lat': 55.677671,
    'lng': 37.562595}],
  'id': '778681'},
 'response_url': None,
 'sort_point_distance': None

### Getting the job data from HeadHunter

In [9]:
#  Defining the function

import requests
import pandas as pd



def get_headhunter_vacancies(keywords='', city_id='1', per_page=100, max_pages=1, date_from='2024-01-20', professional_role = ''):
    base_url = "https://api.hh.ru/vacancies"

    params = {
        "text": keywords,         # Ключевые слова в вакансии (OR между ними)
        'per_page': per_page,      # Кол-во вакансий на 1 странице
        'area': city_id,
        'date_from': date_from,
        # 'date_to': date_to,
        'professional_role': professional_role
    }

    data = {
        "vacancy_name": [],
        "department": [],
        "area_id": [],
        "area_name": [],
        "salary": [],
        "published_at": [],
        "created_at": [],
        "archived": [],
        "employer_id": [],  # Добавлено поле для employer_id
        "employer_name": [],  # Добавлено поле для employer_id
        "employer_link": [],  # Добавлено поле для employer_id
        "city": [],
        "schedule": [],
        "professional_roles": [],
        "working_time_modes": [],
        "experience": [],
        "employment": [],
        "alternate_url":[],
        "description":[]
    }



    for page in range(max_pages):
        params['page'] = page
        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            vacancies = response.json()['items']
            for vacancy in vacancies:
                data["vacancy_name"].append(vacancy.get('name', ''))
                data["department"].append(vacancy.get('department', ''))
                data["area_id"].append(vacancy.get('area', {}).get('id', ''))
                data["area_name"].append(vacancy.get('area', {}).get('name', ''))
                data["salary"].append(vacancy.get('salary', ''))
                data["published_at"].append(vacancy.get('published_at', ''))
                data["created_at"].append(vacancy.get('created_at', ''))
                data["archived"].append(vacancy.get('archived', ''))
                data["employer_id"].append(vacancy.get('employer', {}).get('id', ''))
                data["employer_name"].append(vacancy.get('employer', {}).get('name', ''))
                data["employer_link"].append(vacancy.get('employer', {}).get('alternate_url', ''))
                data["city"].append(city_id)
                data["schedule"].append(vacancy.get('schedule', {}).get('name', ''))
                data["professional_roles"].append(vacancy.get('professional_roles', ''))
                data["working_time_modes"].append(vacancy.get('working_time_modes', ''))
                data["experience"].append(vacancy.get('experience', {}).get('name', ''))
                data["employment"].append(vacancy.get('employment', {}).get('name', ''))
                data["alternate_url"].append(vacancy.get('alternate_url',''))
                data["description"].append(vacancy.get('description',''))


        else:
            print(f"Error {response.status_code}: {response.text}")

    df = pd.DataFrame(data)
    return df


In [10]:

# Using the function
keywords = ""       # "аналитик|python-разработчик" 
city_id = '1'       # '1' - код для Москвы, можно добавить другие коды городов
max_pages = 20

final_df = pd.DataFrame()

for i in range(1, 175):
    professional_role = i
    try:
        vacancies_data = get_headhunter_vacancies(keywords=keywords, city_id=city_id, max_pages=max_pages, professional_role=i)
        final_df = pd.concat([final_df, vacancies_data], ignore_index=True)
    except:
        continue



In [11]:
print( final_df.vacancy_name.count() )
final_df.head(2)

75064


,vacancy_name,department,area_id,area_name,salary,published_at,created_at,archived,employer_id,employer_name,employer_link,city,schedule,professional_roles,working_time_modes,experience,employment,alternate_url,description
0,Продюсер мероприятий,None,1,Москва,None,2024-01-22T15:05:47+0300,2024-01-22T15:05:47+0300,False,2460946,Самокат (ООО Умный ритейл),https://hh.ru/employer/2460946,1,Полный день,"[{'id': '1', 'name': 'Event-менеджер'}]",[],От 1 года до 3 лет,Полная занятость,https://hh.ru/vacancy/92023674,
1,Помощник организатора мероприятий / ивент мене...,None,1,Москва,"{'from': 40000, 'to': 50000, 'currency': 'RUR'...",2024-01-22T04:46:09+0300,2024-01-22T04:46:09+0300,False,1291956,Alexgrim Studio,https://hh.ru/employer/1291956,1,Полный день,"[{'id': '1', 'name': 'Event-менеджер'}]",[],Нет опыта,Стажировка,https://hh.ru/vacancy/91976869,


#### Data cleaning

In [12]:
# create a copy of your dataframe to avoid requesting HH too often
vacancies_full = final_df

In [13]:
# reseting index
vacancies_full = vacancies_full.reset_index(drop=True)
vacancies_full.tail()

,vacancy_name,department,area_id,area_name,salary,published_at,created_at,archived,employer_id,employer_name,employer_link,city,schedule,professional_roles,working_time_modes,experience,employment,alternate_url,description
75059,Инженер СМР,None,1,Москва,"{'from': 80000, 'to': 85000, 'currency': 'RUR'...",2024-01-22T11:33:12+0300,2024-01-22T11:33:12+0300,False,1455718,НПК МЕДИАНА-ФИЛЬТР,https://hh.ru/employer/1455718,1,Полный день,"[{'id': '174', 'name': 'Инженер ПНР'}]",[],От 1 года до 3 лет,Полная занятость,https://hh.ru/vacancy/88818927,
75060,Производитель работ / Site manager,None,1,Москва,None,2024-01-22T10:54:53+0300,2024-01-22T10:54:53+0300,False,1539265,МДИ2Б,https://hh.ru/employer/1539265,1,Полный день,"[{'id': '174', 'name': 'Инженер ПНР'}]",[],От 1 года до 3 лет,Полная занятость,https://hh.ru/vacancy/91920403,
75061,Инженер по организации работ,None,1,Москва,"{'from': 70000, 'to': 75000, 'currency': 'RUR'...",2024-01-22T11:33:57+0300,2024-01-22T11:33:57+0300,False,1455718,НПК МЕДИАНА-ФИЛЬТР,https://hh.ru/employer/1455718,1,Полный день,"[{'id': '174', 'name': 'Инженер ПНР'}]",[],От 1 года до 3 лет,Полная занятость,https://hh.ru/vacancy/88819002,
75062,Инженер ПНР ЛОС,None,1,Москва,"{'from': 100000, 'to': 150000, 'currency': 'RU...",2024-01-22T16:21:33+0300,2024-01-22T16:21:33+0300,False,9637465,ЭКОСТАНДАРТ,https://hh.ru/employer/9637465,1,Полный день,"[{'id': '174', 'name': 'Инженер ПНР'}]",[],От 1 года до 3 лет,Полная занятость,https://hh.ru/vacancy/91946835,
75063,Ведущий инженер отдела пуско-наладочных работ,None,1,Москва,None,2024-01-21T15:34:22+0300,2024-01-21T15:34:22+0300,False,3251101,Турбохолод,https://hh.ru/employer/3251101,1,Полный день,"[{'id': '174', 'name': 'Инженер ПНР'}]",[],От 3 до 6 лет,Полная занятость,https://hh.ru/vacancy/91553042,


In [14]:

# Преобразование поля salary в отдельные колонки
df_salary = pd.json_normalize(vacancies_full['salary']).reset_index()
vacancies_full = pd.concat([vacancies_full, df_salary], axis=1, ignore_index=False)

# Преобразование поля professional_roles в отдельные колонки
df_roles = pd.json_normalize(vacancies_full['professional_roles'].explode())
vacancies_full = pd.concat([vacancies_full, df_roles], axis=1).rename(columns={'id':'role_id'})

# # # adding professional roles
vacancies_full = vacancies_full.merge(prof_roles_df,  
                                    how='left',
                                    left_on='role_id',
                                    right_on='role_id',
                                    copy=False,
                                    indicator = 'x_roles'
                                    )

vacancies_full['published_at'] = pd.to_datetime(vacancies_full.published_at)
vacancies_full['created_at']   = pd.to_datetime(vacancies_full.created_at)
# # 
vacancies_full['from'] = vacancies_full['from'].fillna(0).apply(lambda x: int(x) )
vacancies_full['to']   = vacancies_full['to'].fillna(0).apply(lambda x: int(x) )

vacancies_full = vacancies_full.drop(columns=['salary', 'professional_roles','working_time_modes', 'parent_profession_id', 'parent_profession_name',
                                                'archived','accept_incomplete_resumes','is_default','x_roles','department'])

vacancies_full = vacancies_full.drop_duplicates()


In [15]:
# creating a salary estimate (an average between from and to )
vacancies_full['salary_middle'] = vacancies_full.apply(lambda x: sum( [x['from'],x['to']]  ) / 2 if x['from'] > 0 and x['to'] > 0 else max(x['from'],x['to']) , axis=1)

In [16]:
# check for duplicates and columns that you don' need
vacancies_full.head().T

,0,1,2,3,4
vacancy_name,Продюсер мероприятий,Помощник организатора мероприятий / ивент мене...,Менеджер по организации мероприятий,Event-менеджер в свадебное агентство,Event-менеджер (удаленка)
area_id,1,1,1,1,1
area_name,Москва,Москва,Москва,Москва,Москва
published_at,2024-01-22 15:05:47+03:00,2024-01-22 04:46:09+03:00,2024-01-22 18:16:25+03:00,2024-01-20 13:29:47+03:00,2024-01-21 08:29:56+03:00
created_at,2024-01-22 15:05:47+03:00,2024-01-22 04:46:09+03:00,2024-01-22 18:16:25+03:00,2024-01-20 13:29:47+03:00,2024-01-21 08:29:56+03:00
employer_id,2460946,1291956,41903,5763554,5267544
employer_name,Самокат (ООО Умный ритейл),Alexgrim Studio,Специальные системы и технологии,Свадебное агентство Soulmates,Гудфуд
employer_link,https://hh.ru/employer/2460946,https://hh.ru/employer/1291956,https://hh.ru/employer/41903,https://hh.ru/employer/5763554,https://hh.ru/employer/5267544
city,1,1,1,1,1
schedule,Полный день,Полный день,Полный день,Полный день,Гибкий график


In [17]:
vacancies_full.shape

(75064, 23)

### Descriptive statistics

In [18]:
# count of categories
vacancies_full.name.value_counts().to_frame()

,name
"Менеджер по маркетингу, интернет-маркетолог",2001
"Менеджер по продажам, менеджер по работе с клиентами",2000
Бухгалтер,2000
"Инженер-конструктор, инженер-проектировщик",2000
"Продавец-консультант, продавец-кассир",2000
...,...
Агроном,12
Директор юридического департамента (CLO),9
Зоотехник,9
Бортпроводник,8


In [19]:
vacancies_full.describe(include='object')

,vacancy_name,area_id,area_name,employer_id,employer_name,employer_link,city,schedule,experience,employment,alternate_url,description,currency,gross,role_id,name,role_name
count,75064,75064,75064,75064,75064,75064,75064,75064,75064,75064,75064,75064,55679,55651,75064,75064,75064
unique,48355,1,1,23793,23448,23793,1,5,4,5,75055,1,5,2,174,174,174
top,Офис-менеджер,1,Москва,816144,ВкусВилл,https://hh.ru/employer/816144,1,Полный день,От 1 года до 3 лет,Полная занятость,https://hh.ru/vacancy/90632398,,RUR,False,68,"Менеджер по маркетингу, интернет-маркетолог","Менеджер по маркетингу, интернет-маркетолог"
freq,407,75064,75064,1135,1135,1135,75064,53806,35549,70912,2,75064,55437,40771,2001,2001,2001


In [ ]:
vacancies_full.describe()

In [20]:
# table for analysis 1

import numpy as np 

vacancies_full[(vacancies_full['salary_middle'] != 0)].groupby(['role_name','experience', 'employment']).agg(
    mean=('salary_middle', 'mean'),
    std=('salary_middle', 'std'),
    min=('salary_middle', 'min'),
    perc_25=('salary_middle', lambda x: np.percentile(x, q=0.25)),
    median=('salary_middle', lambda x: np.percentile(x, q=0.5)),
    perc_75=('salary_middle', lambda x: np.percentile(x, q=0.75)),
    max=('salary_middle', 'max'),
    count=('salary_middle', 'count')
).reset_index().to_csv('/Users/igorrudakov/Desktop/job_data_aggr.csv', encoding='utf-8')



In [147]:
# if you need to save current file (checkpoint)
# vacancies_full.to_csv('/Users/igorrudakov/Desktop/last_download_hh.csv')

### Saving first ready to use table into the DataBase [in progress]

In [24]:
vacancies_full[vacancies_full.role_id == '10']

,vacancy_name,area_id,area_name,published_at,created_at,employer_id,employer_name,employer_link,city,schedule,...,description,index,from,to,currency,gross,role_id,name,role_name,salary_middle
14175,Ассистент отдела Retail,1,Москва,2024-01-22 09:48:55+03:00,2024-01-22 09:48:55+03:00,4173118,SuperStep,https://hh.ru/employer/4173118,1,Полный день,...,,5057,90000,0,RUR,False,10,Аналитик,Аналитик,90000.0
14179,Аналитик (удаленно),1,Москва,2024-01-22 12:07:01+03:00,2024-01-22 12:07:01+03:00,1419273,Работут,https://hh.ru/employer/1419273,1,Удаленная работа,...,,5058,120000,120000,RUR,False,10,Аналитик,Аналитик,120000.0
14183,Аналитик (Junior),1,Москва,2024-01-22 10:55:24+03:00,2024-01-22 10:55:24+03:00,2804795,Мегамаркет,https://hh.ru/employer/2804795,1,Удаленная работа,...,,5059,0,0,NaN,NaN,10,Аналитик,Аналитик,0.0
14187,Младший аналитик (IT-компания),1,Москва,2024-01-22 11:46:35+03:00,2024-01-22 11:46:35+03:00,11632,DSM Group,https://hh.ru/employer/11632,1,Полный день,...,,5060,0,54500,RUR,False,10,Аналитик,Аналитик,54500.0
14191,Младший аналитик,1,Москва,2024-01-22 18:31:16+03:00,2024-01-22 18:31:16+03:00,11580,Фармстандарт,https://hh.ru/employer/11580,1,Полный день,...,,5061,0,0,NaN,NaN,10,Аналитик,Аналитик,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18607,Ведущий консультант-аналитик 1С (регламентиров...,1,Москва,2024-01-23 09:47:34+03:00,2024-01-23 09:47:34+03:00,142514,Рамакс Интернейшнл,https://hh.ru/employer/142514,1,Удаленная работа,...,,6165,0,0,NaN,NaN,10,Аналитик,Аналитик,0.0
18611,Системный аналитик по 1С УНФ,1,Москва,2024-01-21 16:24:00+03:00,2024-01-21 16:24:00+03:00,1035394,"Красное & Белое, розничная сеть",https://hh.ru/employer/1035394,1,Полный день,...,,6166,0,0,NaN,NaN,10,Аналитик,Аналитик,0.0
18615,Ведущий консультант-аналитик 1С (блок Аренда),1,Москва,2024-01-23 09:46:58+03:00,2024-01-23 09:46:58+03:00,142514,Рамакс Интернейшнл,https://hh.ru/employer/142514,1,Удаленная работа,...,,6167,0,0,NaN,NaN,10,Аналитик,Аналитик,0.0
18619,Старший консультант-аналитик 1С (регламентиров...,1,Москва,2024-01-23 09:46:38+03:00,2024-01-23 09:46:38+03:00,142514,Рамакс Интернейшнл,https://hh.ru/employer/142514,1,Удаленная работа,...,,6168,0,0,NaN,NaN,10,Аналитик,Аналитик,0.0


### Getting vaccancy description from url

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

browser = webdriver.Chrome()
browser.maximize_window()

In [27]:
# Defining the function
def get_hh_info(link_str):
    
    data = {
        'vacancy_title':[],
        'vacancy_description':[]
    }
    try:
        browser.get(link_str)        
        sleep(2)
        try:
            a = browser.find_element("xpath", '//div[@class="vacancy-description"]').text
        except:
            a = 'no_data'
        data['vacancy_description'].append(a)

        try:
            b = browser.find_element("xpath", '//div[@class="vacancy-title"]').text
        except:
            b = 'no_data'
        data['vacancy_title'].append(b)
        if len(browser.window_handles) > 1:
            browser.close()
        return data
    except:
        return 'no_data'

In [25]:
# getting only vacancies with salary 
# also choosing ONE SPECIFIC professional role - because it takes long to get the vacancy description data 
# you can do more - but just join them later
target_vaccancies = vacancies_full[ (vacancies_full['salary_middle'] > 0) & (vacancies_full['role_id'] == '10') ] # 10 for analysts
target_vaccancies.vacancy_name.count()

362

In [28]:
#  Launching the func
from tqdm import tqdm
tqdm.pandas(desc="Processing")
target_vaccancies['url_description'] = target_vaccancies.alternate_url.progress_apply(lambda x: get_hh_info(x) )

/var/folders/z8/hdkxtn_15wx2h07f409mskwm0000gn/T/ipykernel_2698/659664805.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_vaccancies['url_description'] = target_vaccancies.alternate_url.apply(lambda x: get_hh_info(x) )


In [34]:
target_vaccancies[target_vaccancies.url_description == 'no_data']

,vacancy_name,area_id,area_name,published_at,created_at,employer_id,employer_name,employer_link,city,schedule,...,index,from,to,currency,gross,role_id,name,role_name,salary_middle,url_description


In [321]:
description['description_flag'] = vacancies_w_salary.url_description.apply(lambda x: 1 if x != 'no_data' else 0)

/var/folders/z8/hdkxtn_15wx2h07f409mskwm0000gn/T/ipykernel_69000/47653565.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacancies_w_salary['description_flag'] = vacancies_w_salary.description.apply(lambda x: 1 if x != 'no_data' else 0)


In [317]:
# vacancies_w_salary.to_csv('/Users/igorrudakov/Desktop/vacancies_w_salary.csv', encoding='utf-8', index=True)

### Getting company's industry from url

In [ ]:
# Defining the function
# import requests

def get_company_industry(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses

        data = response.json()
        industries = data.get('industries', [])

        if industries:
            return industries[0].get('name', 'no_data')
        else:
            return 'no_data'

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return 'no_data'

# url = 'https://api.hh.ru/employers/11632' # link empoyer


# Example usage
# url = 'https://api.hh.ru/employers/11632'
# industry = get_company_industry(url)
# print(industry)

### Aggregates: Getting counts of vacancies and professional roles

In [177]:

base_url = "https://api.hh.ru/vacancies"

job_counts = []
professional_role_id = []


for prof_role in range(1,175):
    params = {
            # 'area': 1,
            'professional_role': prof_role,
            'per_page': 1
        }
    resp = requests.get(base_url, params=params)
    if resp.status_code == 200:
        response_data = resp.json()
        found_vacancies = response_data.get('found', 0)

        professional_role_id.append(prof_role)
        job_counts.append(found_vacancies)
    else:
        continue

    data = {
    'Area':0,
    'professional_role_id':professional_role_id,
    'job_counts':job_counts
}


# moscow_count_df = pd.DataFrame(moscow_data)
russia_count_df = pd.DataFrame(data)


In [178]:
# russia_count_df.to_csv('/Users/igorrudakov/Desktop/jobcounts_russia.csv', encoding='utf-8')
# moscow_count_df.to_csv('/Users/igorrudakov/Desktop/jobcounts_moscow.csv', encoding='utf-8')


## Extracting skills with Chat GPT

### defining the function

In [390]:
        prompt = '''
                    Identify key technical skills and tasks from the job description. SHORTEN SKILLS and TASKS to a MAXIMUM of 6 words.
                    I need a short standardized instance of task or skill while retaining the information of what needs to be done on a job.
                    I plan to use this prompt on 100 vacancies - the output must be consistent.
                    Ignore subjective skills such as 'Documentation', 'analytical thinking', 'teamwork', and any skills related to work schedule flexibility. Ignore too
                    broad skills and requirements like: 'Data analysis', 'Database querying', 'Analyse data in excel'. Group related skills under one skill, example:
                    scipy, statmodels can be grouped and recorded as 'Python ML libraries'. Do not repeat essentialy the same skills MS-SQL, PL-SQL - all of them is just 'SQL'.
                    Ensure a concise output. ONLY SHOW 20 MOST IMPORTANT SKILLS and TASKS.
                    Return ONLY a TEXT WITH THE SKILLS AND TASKS with  ';' as a delimeter.
                    The output should look like this: 'Python;SQL;Data visualization;building ML models;evaluating promotion/feature effectiveness;A/B testing'
                    '''
prompt = prompt.strip().replace('\n','').replace('            ','')
prompt
            

"Identify key technical skills and tasks from the job description. SHORTEN SKILLS and TASKS to a MAXIMUM of 6 words.I need a short standardized instance of task or skill while retaining the essential information of what needs to be done on a job.I plan to use this prompt on 100 vacancies - the output must be consistent.Ignore subjective skills such as 'Documentation', 'analytical thinking', 'teamwork', and any skills related to work schedule flexibility. Ignore toobroad skills and requirements like: 'Data analysis', 'Database querying', 'Analyse data in excel'. Group related skills under one skill, example:scipy, statmodels can be grouped and recorded as 'Python ML libraries'. Do not repeat essentialy the same skills MS-SQL, PL-SQL - all of them is just 'SQL'.Ensure a concise output. ONLY SHOW 20 MOST IMPORTANT SKILLS and TASKS.Return ONLY a TEXT WITH THE SKILLS AND TASKS with  ';' as a delimeter.The output should look like this: 'Python;SQL;Data visualization;building ML models;evalua

In [391]:
# for testing in a free CGPT
Markdown(description)


Identify key technical skills and tasks from the job description. SHORTEN SKILLS and TASKS to a MAXIMUM of 6 words.I need a short standardized instance of task or skill while retaining the essential information of what needs to be done on a job.I plan to use this prompt on 100 vacancies - the output must be consistent.Ignore subjective skills such as 'Documentation', 'analytical thinking', 'teamwork', and any skills related to work schedule flexibility. Ignore toobroad skills and requirements like: 'Data analysis', 'Database querying', 'Analyse data in excel'. Group related skills under one skill, example:scipy, statmodels can be grouped and recorded as 'Python ML libraries'. Do not repeat essentialy the same skills MS-SQL, PL-SQL - all of them is just 'SQL'.Ensure a concise output. ONLY SHOW 20 MOST IMPORTANT SKILLS and TASKS.Return them in a string format with  ';' as a delimeter.The output should look like this: 'Python;SQL;Data visualization;building ML models;evaluating promotion/feature effectiveness;A/B testing'

In [401]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
import os


# Set your OpenAI GPT-3 API key
load_dotenv()
client = OpenAI()

api_key=os.environ.get("OPENAI_API_KEY"),
openai.api_key = api_key

def extract_key_skills_requirements(vacancy_description, prompt):
    # try:
        # Define the prompt for ChatGPT
        prompt = prompt

        # Call the OpenAI API
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[ {"role": "user","content": f'{prompt}. Vacancy description is: {vacancy_description}.'} ],
            max_tokens=150, 
            temperature=0.4
        )

        # Check if the response contains choices
        # if 'choices' in response and response['choices']:
            # Extract the generated text from the API response
        generated_text = response.choices[0].message.content
        return generated_text
        # else:
            # return "Error: No response choices found."

    # except Exception as e:
        return f"Error: {str(e)}"




#### Checking the Function output

In [402]:
a = extract_key_skills_requirements( txtdesc.strip().replace('\n', '') , prompt)

In [376]:
a

'Data collection and processing;Query optimization and database operations;Identifying trends and patterns;A/B testing and analysis;Data visualization;Python ML libraries.'

In [267]:
from IPython.display import Markdown

Markdown(test_df['vacancy_description'][3])

Обязанности:
Поиск, структурирование и анализ текстовой информации.
Количественный анализ данных в Excel.
Систематизация и статистическая обработка поступающей информации.
Написание отчетов и подготовка презентаций.
Анализ основных проблем, предложение способа их решения.
Взаимодействие с органами власти города Москвы.
Требования:
Высшее образование (юридическое, экономическое, ГМУ, техническое).
Опыт работы в органах исполнительной власти приветствуется.
Владение Word, Power Point.
Экспертное владение Excel.
(VBA, power query, формулы массивов - как преимущество).
Владение Python для анализа данных (библиотека pandas) - как преимущество.
Аналитический склад ума, умение работать в команде, инициативность.
Усидчивость.
Внимательность.
Условия:
Оформление по ТК РФ.
Испытательный срок 3 месяца.
Готовность к ненормированному рабочему дню.
Возможность профессионального роста и карьерного развития.
Ключевые навыки
MS Excel
MS PowerPoint
Консультант плюс
MS Outlook
Структурирование информации
Работа с большим объемом информации
Python
Задайте вопрос работодателю
Он получит его с откликом на вакансию
Где располагается место работы?Какой график работы?Вакансия открыта?Какая оплата труда?Как с вами связаться?Другой вопрос
Адрес
Москва, Баррикадная, Краснопресненская, Улица 1905 года, Большой Предтеченский переулок, 15/8с1
© Яндекс Условия использования
Показать на большой карте

In [278]:
Markdown( txtdesc.strip().replace('\n', '') )

В крупную международную компанию, которая разнимается разработкой IT-решений для B2B сегмента, ищем талантливого и проактивного сотрудника на должность Data analyst.Чем предстоит заниматься:Сбор, обработка и анализ больших объемов данных (из MySQL, Clickhouse, MongoDB, Google Analytics, Ya.Metrica).Написание и оптимизация запросов в базы данных, работа с разными источниками данных.Выявление тенденций и закономерностей в массивах информации, формулирование и проверка гипотез.Проведение и анализ A/B тестов, интерпретация результатов и формулирование дальнейших действий для улучшения ключевых показателей компании.Визуализация данных для наглядности и дальнейшей работы.Какой опыт, знания и качества нужны:Релевантный опыт в аналитике данных от 3 лет;Хорошее знание теории вероятностей, математической статистики и линейной алгебры;Опыт работы с Google Analytics и/или Я.Метрика (создание API для работы с данными из GA, настраивание и разметка событий и т.д.)Уверенное знание SQL (оконные функции, оптимизация запросов);Знание Python, включая библиотеки для анализа данных (numpy/pandas/scipy/statmodels/sklearn и другие);Опыт работы с BI-системами (Power BI);Понимание основных продуктовых метрик (LTV, ARPU, Retention и т.д.);Знание методологии и практический опыт проведения A/B-тестирования;Умение качественно документировать результаты.Английский язык уровня B1.Soft skills: аналитическое мышление, коммуникативные навыки и работа в команде, саморазвитие.Будет большим плюсом:Опыт работы с Apache Airflow или аналогамиОпыт работы с Mongo DB и ClickHouse.Умение пользоваться Gitlab, Confluence, Jira.Что предлагаем:График работы 5/2, гибкое начало рабочего дня;Удаленный формат работы;Оплачиваемый отпуск 28 календарных дней;Гибкий подход к управлению, постановке задач, процессам;Перспективные проекты и продвинутые технологии;Интересные задачи в дружной и большой команде;Отсутствие ненужной бюрократии, возможность реализовывать свои идеи.

In [403]:

pp = '''
Data collection and analysis;SQL;Google Analytics;A/B testing;Data visualization;Python (numpy, pandas, scipy, statmodels, sklearn);Power BI;Product metrics (LTV, ARPU, Retention);English B1;Apache Airflow;MongoDB, ClickHouse;Gitlab, Confluence, Jira.

'''
ll = a.split(';')
ll

['Data collection',
 'Data processing',
 'Data analysis',
 'Query optimization',
 'Trend identification',
 'Hypothesis formulation and testing',
 'A/B testing',
 'Results interpretation',
 'Data visualization',
 'Probability theory knowledge',
 'Statistical knowledge',
 'Linear algebra knowledge',
 'Google Analytics experience',
 'Ya.Metrica experience',
 'SQL proficiency',
 'Python proficiency',
 'BI system experience',
 'Product metric understanding',
 'A/B testing methodology knowledge',
 'Documentation skills.']

In [394]:
# 0,9
pp = '''
Data collection and analysis;SQL;Google Analytics;A/B testing;Data visualization;Python (numpy, pandas, scipy, statmodels, sklearn);Power BI;Product metrics (LTV, ARPU, Retention);English B1;Apache Airflow;MongoDB, ClickHouse;Gitlab, Confluence, Jira.

'''
ll = a.split(';')
ll

['Data analysis',
 'Database querying',
 'A/B testing',
 'Data visualization',
 'Python',
 'SQL',
 '']

### Preparing the dataset

In [294]:
# Only taking full time jobs
df_for_gpt = target_vaccancies[ (target_vaccancies.schedule == 'Полный день')&( target_vaccancies.employment == 'Полная занятость') ].reset_index(drop=True)
# 
# unpacking descriptions
df_description = pd.json_normalize(df_for_gpt['url_description']).reset_index()
df_for_gpt = pd.concat([df_for_gpt, df_description], axis=1, ignore_index=False)
df_for_gpt = df_for_gpt.drop(columns=['url_description','index','vacancy_title'])

# transforming description into string
df_for_gpt['vacancy_description'] = df_for_gpt.vacancy_description.apply(lambda x: x[0].strip().replace('\n', ''))

### Testing 

In [404]:
test_df = df_for_gpt[['vacancy_name', 'vacancy_description']].head(10)

In [406]:
test_df['output'] = test_df['vacancy_description'].progress_apply(lambda x: extract_key_skills_requirements(x, prompt) )

Processing: 100%|██████████| 10/10 [00:15<00:00,  1.53s/it]


In [407]:
test_df['output']

0    1C reporting;Excel reporting;Report and presen...
1    Data processing;Econometric modeling;SQL query...
2    data analysis;MS Excel;analytical skills;sales...
3    Data analysis;Excel;Presentation preparation;P...
4    Товарная матрица;Анализ продаж и остатков;Рота...
5    Data analysis;Data visualization;Presentation ...
6    Control of applications, invoices; work with p...
7          Excel;Word;Outlook;Powerpoint;R-Тариф;ЭТРАН
8    Data Analyst;Data analysis;Data visualization;...
9    Data/text mining;Linguistic analysis;Data coll...
Name: output, dtype: object

In [412]:
test_df['output'][6].split(';')

['Control of applications, invoices',
 ' work with primary accounting documentation',
 ' contract management',
 ' accounting of completed work',
 ' 1C ERP',
 ' 1C UPP',
 ' payment account management',
 ' documentation management',
 ' database work',
 ' report preparation skills.']

In [312]:
test_df['output'][0].replace("'",'"')

'{"1C": "software", "Excel": "software", "PowerPoint": "software", "Power BI": "software", "MS Office": "software", "communication skills": "communication", "multitasking": "general", "analytical thinking": "general", "English": "language", "Russian": "language"}'

In [313]:
import json

# Your string
input_string = test_df['output'][0].replace("'",'"')
# input_string
# Parse the string into a dictionary
parsed_dict = json.loads(input_string)

# Now 'parsed_dict' is a Python dictionary
parsed_dict


In [314]:
parsed_dict

{'1C': 'software',
 'Excel': 'software',
 'PowerPoint': 'software',
 'Power BI': 'software',
 'MS Office': 'software',
 'communication skills': 'communication',
 'multitasking': 'general',
 'analytical thinking': 'general',
 'English': 'language',
 'Russian': 'language'}

In [316]:
# test_df['output'] = test_df['output'].apply(lambda x: x.replace("'",'"') )
test_df['output'] = test_df['output'].apply(lambda x: json.loads(x) )
# test_df['output']

JSONDecodeError: Extra data: line 1 column 79 (char 78)

In [321]:
json.loads( test_df['output'][2] )

JSONDecodeError: Extra data: line 1 column 79 (char 78)

In [336]:
test_df.iloc[2,:].output.strip().replace('\n','') 

'{"аналитическая поддержка продаж коммерческой дирекции": "analytical support"},{"обработка источников и анализ полученной информации": "data analysis"},{"помощь в подготовке материалов на совещания Коммерческих директоров": "meeting preparation"},{"выполнение оперативных запросов от Коммерческих директоров, связанных с аналитикой продаж": "sales analytics"},{"оконченное высшее образование": "education"},{"уверенный пользователь MS Excel": "MS Excel"},{"аналитические навыки, умение делать выводы из больших массивов данных": "data analysis"},{"офис в центре Москвы": "location"},{"работа в крупном развивающемся Холдинге с перспективами профессионального развития": "company development"},{"официальное оформление с первого дня": "employment"},{"график работы 5/2": "work schedule"}'

In [334]:
json.loads('{"аналитическая поддержка продаж коммерческой дирекции": "analytical support"},{"обработка источников и анализ полученной информации": "data analysis"},{"помощь в подготовке материалов на совещания Коммерческих директоров": "meeting preparation"},{"выполнение оперативных запросов от Коммерческих директоров, связанных с аналитикой продаж": "sales analytics"},{"оконченное высшее образование": "education"},{"уверенный пользователь MS Excel": "MS Excel"},{"аналитические навыки, умение делать выводы из больших массивов данных": "data analysis"},{"офис в центре Москвы": "location"},{"работа в крупном развивающемся Холдинге с перспективами профессионального развития": "company development"},{"официальное оформление с первого дня": "employment"},{"график работы 5/2": "work schedule"}')

JSONDecodeError: Extra data: line 1 column 79 (char 78)